In [1]:
import pickle
import numpy as np
import numpy as np
import cv2
import torch
from torch import nn
import matplotlib.pyplot as plt

/root/fashion-effnet/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_PATH = "../trained_models/watches_acc_96"

In [3]:
from PIL import Image
import numpy as np
import cv2
def predict(modelpath, image, transforms, encoder):
    print("Loading model.pth...")
    model = torch.load(modelpath + '/model.pth')
    model.eval()
    if(isinstance(image,np.ndarray)):
      image = Image.fromarray(image)
    if(transforms!=None):
        image = transforms(image)
    data = image.expand(1,-1,-1,-1)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Forward Passing...")
    data = data.type(torch.FloatTensor).to(device)
    sm = nn.Softmax(dim = 1)
    output = model(data)
    output = sm(output)
    print("Predicting Label...")
    _, preds = torch.max(output, 1)

    pred_label = encoder[preds.cpu().detach().numpy()[0]]
    return pred_label
def prediction_bar(output,encoder):
    output = output.cpu().detach().numpy()
    a = output.argsort()
    a = a[0]
    
    size = len(a)
    if(size>5):
        a = np.flip(a[-5:])
    else:
        a = np.flip(a[-1*size:])
    prediction = list()
    clas = list()
    for i in a:
      prediction.append(float(output[:,i]*100))
      clas.append(str(i))
    for i in a:
        print('Class: {} , confidence: {}'.format(encoder[int(i)],float(output[:,i]*100)))
    plt.bar(clas,prediction)
    plt.title("Confidence score bar graph")
    plt.xlabel("Confidence score")
    plt.ylabel("Class number")
 
def img_plot(image,inv_normalize = None):
    if(inv_normalize!=None):
        image = inv_normalize(image)
    image = image.cpu().numpy().transpose(1,2,0)
    plt.imshow(image)
    plt.show()


In [4]:
image = cv2.imread('../sample_images/watch_codecoco.jpg')

In [5]:
print("=======Phase 1: Parent Class Prediction========")
modelpath = "../trained_models/meta"

print("Loading pkl files...")
with open(modelpath + "/bin/encoder.pickle", 'rb') as fr:
    encoder = pickle.load(fr)
with open(modelpath + "/bin/test_transforms.pickle", 'rb') as fr:
    transforms = pickle.load(fr)
with open(modelpath + "/bin/inv_normalize.pickle", 'rb') as fr:
    inv_normalize = pickle.load(fr)
print("ENCODER: ", encoder)
parent_pred_label = predict(modelpath, image, transforms, encoder)

parent_pred_class = parent_pred_label.split('_', 1)[0].replace('.', '') # watch, cosm, perf, etc.

print("Parent class prediction: ", parent_pred_class)
child_modelpath = "../trained_models/" + parent_pred_class
print("=======Phase 2: Child Class Prediction========")

print("Loading pkl files...")
with open(child_modelpath + "/bin/encoder.pickle", 'rb') as fr:
    encoder = pickle.load(fr)
with open(child_modelpath + "/bin/test_transforms.pickle", 'rb') as fr:
    transforms = pickle.load(fr)
with open(child_modelpath + "/bin/inv_normalize.pickle", 'rb') as fr:
    inv_normalize = pickle.load(fr)
print("ENCODER: ", encoder)
child_pred_label = predict(child_modelpath, image, transforms, encoder)


print("Final Prediction: ", child_pred_label)

=======Phase 1: Parent Class Prediction========
Loading pkl files...
ENCODER:  {0: 'accessories', 1: 'bags', 2: 'cosmetics', 3: 'perfume', 4: 'wallets', 5: 'watches'}
Loading model.pth...
Forward Passing...
Predicting Label...
Parent class prediction:  accessories
=======Phase 2: Child Class Prediction========
Loading pkl files...
ENCODER:  {0: 'access_belts', 1: 'access_buttonjewelry', 2: 'access_cococrush', 3: 'access_gloves', 4: 'access_hairclip', 5: 'access_headwear', 6: 'access_paperbag', 7: 'access_scarves'}
Loading model.pth...
Forward Passing...
Predicting Label...
Final Prediction:  access_cococrush
